In [2]:
from hyperopt import hp

# -10 ~ 10까지 1간격을 가지는 입력 변수 x 집합값과 -15 ~ 15까지 1간격을 가지는 입력 변수 y 집합값 설정.
search_space = {'x' : hp.quniform('x',-10,10,1),'y': hp.quniform('y',-15,15,1)}

In [3]:
search_space

{'x': <hyperopt.pyll.base.Apply at 0x12fb94dd0>,
 'y': <hyperopt.pyll.base.Apply at 0x12fd9ca10>}

In [4]:
from hyperopt import STATUS_OK

# 목적 함수를 생성. 입력 변수값과 입력 변수 검색 범위를 가지는 딕셔너리를 인자로 받고, 특정 값을 반환
def objective_func(search_space):
    x = search_space['x']
    y = search_space['y']
    retval = x**2 - 20*y
    
    return retval # return {'loss':retval,'status':STATUS_OK}

In [5]:
from hyperopt import fmin,tpe,Trials
import numpy as np

# 입력 결괏값을 저장한 Trials 객체 값 생성
trial_val = Trials()

# 목적 함수의 최솟값을 반환하는 최적 입력 변숫값을 5번의 입력값 시도(max_evals=5)로 찾아냄.
best_01 = fmin(fn=objective_func,space=search_space,algo=tpe.suggest,max_evals=5,trials=trial_val
               ,rstate=np.random.default_rng(seed=0))

print('best : ',best_01)

100%|██████████████████████| 5/5 [00:00<00:00, 977.69trial/s, best loss: -224.0]
best :  {'x': -4.0, 'y': 12.0}


In [7]:
trial_val = Trials()
# max_evals=20으로 재 테스트 
best_02 = fmin(fn=objective_func,space=search_space,algo=tpe.suggest,max_evals=20,trials=trial_val
              ,rstate=np.random.default_rng(seed=0))

print('best :',best_02)

100%|████████████████████| 20/20 [00:00<00:00, 973.90trial/s, best loss: -296.0]
best : {'x': 2.0, 'y': 15.0}


In [8]:
trial_val

+ HyperOpt 수행 시 적용된 입력 값들과 목적 함수 반환값 보기

In [9]:
# fmin( )에 인자로 들어가는 Trials 객체의 result 속성에 파이썬 리스트로 목적 함수 반환값들이 저장됨
# 리스트 내부의 개별 원소는 {'loss':함수 반환값, 'status':반환 상태값} 와 같은 딕셔너리임. 
print(trial_val.results)

[{'loss': -64.0, 'status': 'ok'}, {'loss': -184.0, 'status': 'ok'}, {'loss': 56.0, 'status': 'ok'}, {'loss': -224.0, 'status': 'ok'}, {'loss': 61.0, 'status': 'ok'}, {'loss': -296.0, 'status': 'ok'}, {'loss': -40.0, 'status': 'ok'}, {'loss': 281.0, 'status': 'ok'}, {'loss': 64.0, 'status': 'ok'}, {'loss': 100.0, 'status': 'ok'}, {'loss': 60.0, 'status': 'ok'}, {'loss': -39.0, 'status': 'ok'}, {'loss': 1.0, 'status': 'ok'}, {'loss': -164.0, 'status': 'ok'}, {'loss': 21.0, 'status': 'ok'}, {'loss': -56.0, 'status': 'ok'}, {'loss': 284.0, 'status': 'ok'}, {'loss': 176.0, 'status': 'ok'}, {'loss': -171.0, 'status': 'ok'}, {'loss': 0.0, 'status': 'ok'}]


In [12]:
# Trials 객체의 vals 속성에 {'입력변수명':개별 수행 시마다 입력된 값 리스트} 형태로 저장됨.
print(trial_val.vals)

{'x': [-6.0, -4.0, 4.0, -4.0, 9.0, 2.0, 10.0, -9.0, -8.0, -0.0, -0.0, 1.0, 9.0, 6.0, 9.0, 2.0, -2.0, -4.0, 7.0, -0.0], 'y': [5.0, 10.0, -2.0, 12.0, 1.0, 15.0, 7.0, -10.0, 0.0, -5.0, -3.0, 2.0, 4.0, 10.0, 3.0, 3.0, -14.0, -8.0, 11.0, -0.0]}


In [13]:
import pandas as pd

# result에서 loss 키값에 해당하는 밸류들을 추출하여 list로 생성.
losses = [loss_dict['loss'] for loss_dict in trial_val.results]

# DataFrame으로 생성
result_df = pd.DataFrame({'x':trial_val.vals['x'],
                         'y':trial_val.vals['y'],
                         'losses': losses
                         })
result_df

,x,y,losses
0,-6.0,5.0,-64.0
1,-4.0,10.0,-184.0
2,4.0,-2.0,56.0
3,-4.0,12.0,-224.0
4,9.0,1.0,61.0
5,2.0,15.0,-296.0
6,10.0,7.0,-40.0
7,-9.0,-10.0,281.0
8,-8.0,0.0,64.0
9,-0.0,-5.0,100.0


### HyperOpt를 XGBoost 하이퍼 파라미터 튜닝에 적용

In [18]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings('ignore')

dataset = load_breast_cancer()

cancer_df = pd.DataFrame(dataset.data,columns=dataset.feature_names)
cancer_df['target'] = dataset.target 

X_feature = cancer_df.iloc[:,:-1]
y_label = cancer_df.iloc[:,-1]

X_train,X_test,y_train,y_test = train_test_split(X_feature,y_label,test_size=0.2)

X_tr,X_val,y_tr,y_val = train_test_split(X_train,y_train,test_size=0.1)


In [27]:
from hyperopt import hp

# max_depth는 5에서 20까지 1간격으로 min_child_weight는 1에서2까지 1간격으로
# colsample_bytreesms 0.5에서 1사이,learning_rate는 0.01에서 0.2 정규 분포된 값으로 검색

xgb_search_space = {'max_depth': hp.quniform('max_depth',5,20,1),
                    'min_child_weight': hp.quniform('min_child_weight',1,2,1),
                    'colsample_bytree': hp.uniform('colsample_bytree',0.5,1),
                    'learning_rate': hp.uniform('learning_rate',0.01,0.2)}

In [28]:
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from hyperopt import STATUS_OK

# fmin()에서 입력된 search_space값으로 입력된 모든 값은 실수형임. 
# XGBClassifier의 정수형 하이퍼 파라미터는 정수형 변환을 해줘야 함. 
# 정확도는 높은 수록 더 좋은 수치임. -1* 정확도를 곱해서 큰 정확도 값일 수록 최소가 되도록 변환
def objective_func(search_space):
    # 수행시간 절약을 위해 n_estimators는 100으로 축소
    xgb_clf = XGBClassifier(n_estimators=100,max_depth=int(search_space['max_depth']),
                            min_child_weight = int(search_space['min_child_weight']),
                            learning_rate=search_space['learning_rate'],
                            colsample_bytree = search_space['colsample_bytree'],
                            eval_metric='logloss')
    
    accuracy = cross_val_score(xgb_clf,X_train,y_train,scoring='accuracy',cv=3)
    
    # accuracy 는 cv=3개수만큼의 정확도 결과를 가지므로 이를 평균해서 반환하되 -1을 곱해줌
    
    return {'loss':-1*np.mean(accuracy),'status':STATUS_OK}


In [29]:
from hyperopt import fmin,tpe,Trials

trial_val =Trials()
best = fmin(fn=objective_func,space=xgb_search_space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trial_val)
print("best :",best)

100%|████████| 50/50 [00:06<00:00,  7.98trial/s, best loss: -0.9626031137446264]
best : {'colsample_bytree': 0.5018690429031908, 'learning_rate': 0.14017335542259288, 'max_depth': 18.0, 'min_child_weight': 2.0}


In [30]:
print('colsample_bytree:{0}, learning_rate:{1}, max_depth:{2}, min_child_weight:{3}'.format(
                        round(best['colsample_bytree'], 5), round(best['learning_rate'], 5),
                        int(best['max_depth']), int(best['min_child_weight'])))

colsample_bytree:0.50187, learning_rate:0.14017, max_depth:18, min_child_weight:2


In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    # ROC-AUC 추가 
    roc_auc = roc_auc_score(y_test, pred_proba)
    print('오차 행렬')
    print(confusion)
    # ROC-AUC print 추가
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [32]:
xgb_wrapper = XGBClassifier(n_estimators = 400,learning_rate=round(best['learning_rate'],5),
                            max_depth = int(best['max_depth']),min_child_weight = int(best['min_child_weight']),
                            colsample_bytree=best['colsample_bytree'])
evals = [(X_tr, y_tr), (X_val, y_val)]
xgb_wrapper.fit(X_tr,y_tr,early_stopping_rounds=50,eval_metric='logloss',eval_set=evals,verbose=True)

preds = xgb_wrapper.predict(X_test)
pred_proba = xgb_wrapper.predict_proba(X_test)[:,1]

get_clf_eval(y_test,preds,pred_proba)

[0]	validation_0-logloss:0.58289	validation_1-logloss:0.59079
[1]	validation_0-logloss:0.49685	validation_1-logloss:0.50267
[2]	validation_0-logloss:0.42976	validation_1-logloss:0.44140
[3]	validation_0-logloss:0.37446	validation_1-logloss:0.39166
[4]	validation_0-logloss:0.33154	validation_1-logloss:0.34315
[5]	validation_0-logloss:0.29349	validation_1-logloss:0.31291
[6]	validation_0-logloss:0.26086	validation_1-logloss:0.27536
[7]	validation_0-logloss:0.23340	validation_1-logloss:0.24933
[8]	validation_0-logloss:0.20978	validation_1-logloss:0.22844
[9]	validation_0-logloss:0.18933	validation_1-logloss:0.20403
[10]	validation_0-logloss:0.17244	validation_1-logloss:0.18444
[11]	validation_0-logloss:0.15712	validation_1-logloss:0.17196
[12]	validation_0-logloss:0.14433	validation_1-logloss:0.15916
[13]	validation_0-logloss:0.13294	validation_1-logloss:0.14992
[14]	validation_0-logloss:0.12300	validation_1-logloss:0.13821
[15]	validation_0-logloss:0.11358	validation_1-logloss:0.12892
[1

[130]	validation_0-logloss:0.01608	validation_1-logloss:0.04531
[131]	validation_0-logloss:0.01602	validation_1-logloss:0.04552
[132]	validation_0-logloss:0.01597	validation_1-logloss:0.04506
[133]	validation_0-logloss:0.01591	validation_1-logloss:0.04532
[134]	validation_0-logloss:0.01586	validation_1-logloss:0.04543
[135]	validation_0-logloss:0.01581	validation_1-logloss:0.04579
[136]	validation_0-logloss:0.01576	validation_1-logloss:0.04581
[137]	validation_0-logloss:0.01571	validation_1-logloss:0.04608
[138]	validation_0-logloss:0.01566	validation_1-logloss:0.04636
오차 행렬
[[41  1]
 [ 1 71]]
정확도: 0.9825, 정밀도: 0.9861, 재현율: 0.9861,    F1: 0.9861, AUC:0.9997


In [33]:
losses = [loss_dict['loss'] for loss_dict in trial_val.results]
result_df = pd.DataFrame({'max_depth': trial_val.vals['max_depth'],
                          'min_child_weight': trial_val.vals['min_child_weight'],
                          'colsample_bytree': trial_val.vals['colsample_bytree'],
                          'learning_rate': trial_val.vals['learning_rate'],
                          'losses': losses
                         }
                        )
result_df

,max_depth,min_child_weight,colsample_bytree,learning_rate,losses
0,10.0,1.0,0.679030,0.027392,-0.942837
1,7.0,2.0,0.639898,0.197092,-0.956024
2,16.0,2.0,0.882734,0.110725,-0.953802
3,10.0,2.0,0.717072,0.135197,-0.955995
4,7.0,1.0,0.840375,0.124859,-0.951595
5,7.0,1.0,0.829519,0.153391,-0.945016
6,17.0,1.0,0.862047,0.033687,-0.942808
7,8.0,1.0,0.852322,0.152206,-0.949402
8,17.0,2.0,0.656299,0.199226,-0.958217
9,19.0,1.0,0.995589,0.029320,-0.940615
